In [135]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
from fuzzywuzzy import fuzz
from rapidfuzz import process, fuzz
from geopy.geocoders import Nominatim
import time
import openpyxl
import concurrent.futures
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from fuzzywuzzy import fuzz
import time
import re
import random
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.namespace import Namespace

# this gets the tables from the list of all Belgian beers, and does some cleaning 

In [ ]:
url = "https://fr.wikipedia.org/wiki/Liste_de_bi%C3%A8res_belges"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

def remove_wikipedia_ref(text):
	return re.sub(r"\[\d+\]", "", text)

def remove_language_tags(text):
	return re.sub(r"\s*\((nl|en)\)", "", text)

def remove_trailing_commas(text):
	count_trailing = re.search(r",+$", text)
	if count_trailing:
		return text[:-len(count_trailing.group())]
	return text

def remove_pour_and_following_text(text): # need to adjust to remove the parenthesis
	if "pour" in text:
		return text.split("pour")[0][:-1]
	return text

dataframes = []
for i, table in enumerate(tables):
    headers = [header.text.strip() for header in table.find_all("th")]
    rows = []
    for row in table.find_all("tr")[1:]: # start at 1 skip header 
        cols = row.find_all(["td", "th"])
        cols = [col.text.strip() for col in cols]
        cols = [remove_wikipedia_ref(col) for col in cols]
        cols = [remove_trailing_commas(col) for col in cols]
        cols = [remove_language_tags(col) for col in cols]
        cols = [remove_pour_and_following_text(col) for col in cols]
        if cols:
            rows.append(cols)
    df = pd.DataFrame(rows, columns=headers)
    dataframes.append(df)
	
beers = pd.concat(dataframes, ignore_index=True)

In [ ]:
beers

# this gets the data from list of Belgian breweries, as well as some additional beer

In [ ]:
url = "https://fr.wikipedia.org/wiki/Liste_de_brasseries_belges"

reponse = requests.get(url)
soup = BeautifulSoup(reponse.text, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

dataframes_breweries = []
for i, table in enumerate(tables):
    headers = [header.text.strip() for header in table.find_all("th")]
    rows = []
    for row in table.find_all("tr")[1:]:
        cols = row.find_all(["td", "th"])
        cols = [col.text.strip() for col in cols]
        cols = [remove_wikipedia_ref(col) for col in cols]
        cols = [remove_trailing_commas(col) for col in cols]
        cols = [remove_language_tags(col) for col in cols]
        cols = cols[:6]
        if cols:
            rows.append(cols)
    df = pd.DataFrame(rows, columns=headers)
    dataframes_breweries.append(df.iloc[:, :4])
    
breweries = pd.concat(dataframes_breweries, ignore_index=True)

# this block just removes some useless columns and rows
breweries = breweries.iloc[:250, :4]

# we need to align the breweries from both datasets with the format of one of the two
for this we will use the transformers -- sentence_transformers
or fuzzy thing, idk which one will work best




In [ ]:
def rename_by_similarity(df_source, col_source, df_reference, col_reference, threshold=70):
    ref_values = df_reference[col_reference].dropna().unique()
    df_source = df_source.copy()

    def get_best_match(value):
        if pd.isna(value) or value == "":
            return value
        match, score, i = process.extractOne(value, ref_values, scorer=fuzz.ratio)
        return match if score >= threshold else value # renaming if above threshold

    df_source[col_source] = df_source[col_source].apply(get_best_match)
    return df_source

# here i'm going to add some coordinates using geopy library

In [ ]:
new = rename_by_similarity(beers, "Brasserie", breweries, "Brasserie")

In [ ]:
parents = []

for index, row in breweries.iterrows():
	if "(" in row["Brasserie"]:
		parents.append(row["Brasserie"].split("(")[1][:-1])
	else:
		parents.append(None)

breweries["Parent"] = parents

In [ ]:
breweries["Brasserie"] = breweries["Brasserie"].str.split('(').str[0].str.strip()

# here i just parse a .txt file i created using excel 

In [ ]:
# this was created by hand in excel, just copied the Parent column and did some =IF(condition) function
# as well as manually modifying the values of the column that were not missing
with open ("parent.txt", "r") as f:
	parents_list = [line.strip() for line in f]

In [ ]:
# now to assign it to the breweries dataset
breweries["Parent"] = parents_list

In [ ]:
# here i remove all parentheses from the "Brasserie" column
for i, row in beers.iterrows():
    if "(" in row["Brasserie"]:
        beers.at[i, "Brasserie"] = row["Brasserie"].split("(")[0].strip()

In [ ]:
breweries.to_csv("breweries.csv", index=False)
new.to_csv("beers.csv", index=False)

# this part of the notebook is about somehow merging the right brewery row to the beer

In [ ]:
# a maybe inefficient approach is to look at the similarity once again, and match that
# turns out this is mostly solved with a left join on the "Brasserie" column
merged_df = beers.merge(breweries, on="Brasserie", how="left")

In [ ]:
# i can save as .csv later, first i will just save as .xlsx and add everything missing manually
# i have to use the openpyxl engine, otherwise i can't encode everything properly
# so all accents etc disappear, which is very annoying
merged_df.to_excel("output.xlsx", index=False, engine="openpyxl")

In [ ]:
beers = pd.read_csv("beers.csv")
breweries = pd.read_csv("breweries.csv")

In [ ]:
beer_names = beers["Bières"].tolist()
brewery_names = beers["Brasserie"].tolist()

In [ ]:
df = pd.read_excel("output.xlsx")

# big issue -> the % is being copied with .explode, use multi column

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
# df['Principales bières'] = df['Principales bières'].fillna("").apply(lambda x: x.split(", ") if x else [])
# df = df.explode('Principales bières')
# df.drop(columns=['Principales bières'], inplace=True)

# here i translate to english

In [ ]:
translation_map = {
    'Liège': 'Liege',
    'Brabant wallon': 'Walloon Brabant',
    'Limbourg': 'Limburg',
    'Anvers': 'Antwerp',
    'Brabant flamand': 'Flemish Brabant',
    'Luxembourg': 'Luxembourg',
    'Namur': 'Namur',
    'West-Flanders': 'West Flanders',
    'Flemish-Brabant': 'Flemish Brabant',
    'Hainaut': 'Hainaut',
    'Flandre-Orientale': 'East Flanders',
    'Flandre-Occidentale': 'West Flanders',
    'Antwerp': 'Antwerp',
    'Bruxelles': 'Brussels',
    'East-Flanders': 'East Flanders',
    'Brabant Wallon': 'Walloon Brabant',
    'Walloon Brabant': 'Walloon Brabant'
}

df["Province"] = df["Province"].apply(lambda x: translation_map.get(x, x))


In [ ]:
df['Province'].unique()

In [ ]:
df.drop_duplicates(subset= 'Bières', inplace=True)
df.drop(columns=['Principales bières'], inplace=True) # this column is not needed anymore, it's hard to use

In [ ]:
beer_names = df["Bières"].tolist()

# here i'm just replacing some names in the brewery column

In [ ]:
Brasserie = df["Brasserie"].tolist()
for i, x in enumerate(Brasserie):
	i = str(i)
	if "par" in i:
		Brasserie[i] = i.split("par")[1].strip()

df["Brasserie"] = Brasserie

# this gets the initial URLs needed for the data scraping

In [ ]:
# here i am going to build a list of search URLs to scrape data from untappd
# what this should allow me to do is fetch the next URL i need, that of the page of each individual beer
# given untappd closed their API, i have to scrape using Proxies, which is a bit slow

template = "https://untappd.com/search?q=X"
search_url = []

for i, beer in enumerate(beer_names):
	search_url.append((template.replace("X", beer).replace(" ", "+"), beer))

In [ ]:
untappd = pd.read_csv("untappd.csv")

In [ ]:
out = list(untappd.itertuples(index=False, name=None))
len(out)

In [ ]:
def find_link_fuzzy(driver, target_text, threshold=70): # threshold is lower than the one we used before, it helps to find more matches
    links = driver.find_elements(By.TAG_NAME, "a")
    best_match = None
    best_score = 0
    for link in links:
        link_text = link.text.strip()
        if link_text:
            score = fuzz.ratio(target_text.lower(), link_text.lower())
            if score > best_score:
                best_score = score
                best_match = link
    if best_match and best_score >= threshold:
        return best_match, best_score
    else:
        return None, 0

# this part of the code is given the initial search URL and the target text, it will try to find the URL of the target text
def get_url_fuzzy(driver, url_tuple):
    driver.get(url_tuple[0])
    time.sleep(random.uniform(1,3)) # add some delay
    try:
        link, score = find_link_fuzzy(driver, url_tuple[1])
        if link:
            url = link.get_attribute("href")
            print(f"match found: '{link.text}' (score: {score})")
            return url
        else:
            print(f"match found for '{url_tuple[1]}'")
            return None
    except Exception as e:
        print(f"error: {e}")
        return None

def get_data(url):
    chrome_options = Options()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    except Exception as e:
        print(f"error: {e}")
        return None, None, None # return None values if driver could not be created
    try:
        driver.get(url)
        print(f"opened: {url}")
        # this part of the code handled the consent form of Untappd. 
        # it is always present, but needs human interaction to be closed
        # selenium can handle it, but it's a bit tricky
        try:
            consent_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Consent"]')
            consent_button.click()
        except Exception as e:
            print(f"cannot find form: {e}")

        time.sleep(random.uniform(1, 3))
        # this gets the rating of the beer
        # you can find the rating in the span element with the class "num"
		# the rating is inside parentheses, so you have to strip them
        try:
            rating_element = driver.find_element(By.CSS_SELECTOR, 'span.num')
            rating_text = rating_element.text
            rating = rating_text.strip('()')
        except Exception as e:
            print(f"error: {e}")
            rating = None
		# same for checkin 
        try:
            checkin_element = driver.find_element(By.CSS_SELECTOR, 'span.count')
            checkin_text = checkin_element.text
            checkin_count = checkin_text.replace(',', '')
        except Exception as e:
            print(f"error: {e}")
            checkin_count = None
		# this part of the code gets the description of the beer
        try:
            # sometimes the description is hidden behind a "Read More" button
			# this part of the code tries to click it
            try:
                read_more_button = WebDriverWait(driver, random.uniform(3, 5)).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.read-more.track-click'))
                )
                read_more_button.click()
            except:
                print("error clicking read more")

            description_element = driver.find_element(By.CSS_SELECTOR, 'div.beer-descrption-read-less')
            description_text = description_element.text
            description = re.sub(r'Show Less.*', '', description_text).strip()
        except Exception as e:
            print(f"error: {e}")
            description = None

    except Exception as e: # when the URL cannot be opened, simply return None 
        print(f"error: {e}")
        rating, checkin_count, description = None, None, None

    finally:
        driver.quit()

    return rating, checkin_count, description

chrome_options = Options() # user agent for my version of chrome
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
a
processed = {beer[0] for beer in out} # out as a global variable

for i in search_url:
    if i[1] in processed: # we can skip if already processed
        continue

    url = get_url_fuzzy(driver, i)
    if url:
        rating, checkin_count, description = get_data(url)
        print(f"{i[1]} done!")
        out.append((i[1], rating, checkin_count, description))
    else: # if no URL found, add None values
        print(f"no URL found: {i[1]}")
        out.append((i[1], None, None, None))

    time.sleep(random.uniform(1, 6)) # this adds some delay between requests, to avoid being blocked

driver.quit()

In [ ]:
untappd = pd.DataFrame(out, columns=["Bières", "Rating", "Check-ins", "Description"])

In [ ]:
untappd

In [ ]:
untappd.drop_duplicates(subset="Bières", inplace=True)

In [ ]:
untappd.to_csv("untappd.csv", index=False)

In [ ]:
len(out)

# now i need to merge the dataframes together, the one with Untappd data, and the big dataset 

In [ ]:
df = df.merge(untappd, on="Bières", how="left")

# adding coordinates 


In [ ]:
geolocator = Nominatim(user_agent="myGeocoder", timeout=3) # have to try using different timeout values

coordinates = []

for index, row in df.iterrows():
    brewery = row["Brasserie"]
    commune = row["Commune"]
    province = row["Province"]

    retries = 3
    for i in range(retries):
        try:
            location = geolocator.geocode(brewery) # first we try getting coords of the brewery (most precise)
            if location:
                coordinates.append((location.latitude, location.longitude))
                break # when found coordinates break loop
            else:
                location = geolocator.geocode(commune) # moving onto commune
                if location:
                    coordinates.append((location.latitude, location.longitude))
                    break
                else:
                    coordinates.append(None) # nothing found, might add manually
                    break
        except Exception as e:
            print(f"Error processing {brewery}: {e}")
            time.sleep(5)
    
    print(index)

df["coordinates"] = coordinates

In [ ]:
df.to_csv("final.csv", index=False)

In [ ]:
for i in df.columns:
	col = df[i]
	if col.isnull().sum() > 0:
		print(i, col.isnull().sum())
	else:
		print(i, 0)

In [ ]:
translation_map = {
        'Bières': 'Beers',
        'Type': 'Type',
        'Teneur en alcool': 'Alcohol Content',
        'Brasserie': 'Brewery',
        'Commune': 'Municipality',
        'Province': 'Province',
        'Parent': 'Parent',
        'Rating': 'Rating',
        'Check-ins': 'Check-ins',
        'Description': 'Description'
    }

df = df.rename(columns=translation_map)

In [142]:
# this function is used to convert the check-ins to a float
def convert_to_float(val):
    val = str(val).replace("+", "")
    if 'M' in val:
        return float(val.replace('M', '')) * 1000000
    return float(val)

df['Check-ins'] = df['Check-ins'].apply(convert_to_float)

In [ ]:
df['Latitude'] = df['coordinates'].apply(lambda x: x[0] if x else None)
df['Longitude'] = df['coordinates'].apply(lambda x: x[1] if x else None)
df.drop(columns=['coordinates'], inplace=True)

In [ ]:
df.to_csv("final.csv", index=False)

In [151]:
df['Municipality'] = df['Municipality'].str.replace(" ", "_")

In [152]:
df

,Beers,Type,Alcohol Content,Brewery,Municipality,Province,Parent,Rating,Check-ins,Description,Latitude,Longitude
0,3 Schténg,"Fermentation haute, Vieille Brune Flamande",6 %,Brasserie Grain d'Orge,Plombières,Liege,NaN,3.15,3652.0,NaN,50.737732,5.961113
1,IV Saison,Saison blonde,"6,5 %",Brasserie de Jandrain-Jandrenouille,Orp-Jauche,Walloon Brabant,NaN,3.64,11522.0,"Bière naturelle blonde 100% malt d'orge, non f...",50.677482,4.977243
2,V Cense,"Fermentation haute, Spéciale","7,5 %",Brasserie de Jandrain-Jandrenouille,Orp-Jauche,Walloon Brabant,NaN,3.53,4705.0,"Bière naturelle ambrée, 100% malt d'orge, non ...",50.677482,4.977243
3,VI Wheat,"Fermentation haute, Blanche",6 %,Brasserie de Jandrain-Jandrenouille,Orp-Jauche,Walloon Brabant,NaN,3.51,6579.0,"Bière naturelle, mélange de malt de froment et...",50.677482,4.977243
4,7-PK,Fermentation haute blonde,7 %,Brasserie Anders!,Halen,Limburg,NaN,3.39,8801.0,7PK is een blond haverbier met een alcoholgeha...,52.958748,5.723328
...,...,...,...,...,...,...,...,...,...,...,...,...
1850,Zulte,"Fermentation haute, Brune","4,7 %",Alken-Maes,NaN,NaN,NaN,NaN,NaN,NaN,50.879329,5.307086
1851,Zwalmse Tripel,Triple,8 %,De Proefbrouwerij,Lochristi,West Flanders,NaN,3.48,933.0,NaN,51.115254,3.816097
1852,Zwarte Pol,Stout,"6,5 %",Brasserie Inter-Pol,Houffalize,Luxembourg,NaN,3.55,401.0,NaN,50.132376,5.788693
1853,Zwet.be,Stout,7 %,De Proefbrouwerij,Lochristi,West Flanders,NaN,NaN,NaN,NaN,51.115254,3.816097


In [167]:
df['Municipality'].isnull().sum()

np.int64(0)

In [173]:
for column in df.columns:
    for i, row in enumerate(df[column]):
        if isinstance(row, str):
            df.at[i, column] = row.replace("\"", "")
        if pd.isna(row):  # This catches both NaN and None
            df.at[i, column] = "Unknown"
            
numeric_columns = ["Check-ins", "Alcohol Content", "Rating"]  # Adjust column names as needed
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # Convert to numbers, NaN if invalid
    df[col] = df[col].fillna(0).astype(int)

In [174]:
df.to_csv("final.csv", index=False)

In [175]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

# Load the CSV
df = pd.read_csv("final.csv")

# Define the RDF graph
g = Graph()

# Define Namespaces
SCHEMA = Namespace("https://schema.org/")
EX = Namespace("http://example.org/")  # Custom namespace for this project

g.bind("schema", SCHEMA)
g.bind("ex", EX)

# Iterate over the DataFrame and add triples
for _, row in df.iterrows():
    # Convert to string and handle missing values
    beer_name = str(row["Beers"]).strip() if pd.notna(row["Beers"]) else "Unknown_Beer"
    brewery_name = str(row["Brewery"]).strip() if pd.notna(row["Brewery"]) else "Unknown_Brewery"
    municipality = str(row["Municipality"]).strip() if pd.notna(row["Municipality"]) else "Unknown_Municipality"
    province = str(row["Province"]).strip() if pd.notna(row["Province"]) else "Unknown_Province"
    
    beer_uri = URIRef(EX + beer_name.replace(" ", "_"))
    brewery_uri = URIRef(EX + brewery_name.replace(" ", "_"))
    municipality_uri = URIRef(EX + municipality.replace(" ", "_"))
    province_uri = URIRef(EX + province.replace(" ", "_"))
    location_uri = URIRef(EX + f"location_{row['Latitude']}_{row['Longitude']}")

    # Define Classes
    g.add((beer_uri, RDF.type, SCHEMA.BreweryProduct))
    g.add((brewery_uri, RDF.type, SCHEMA.Brewery))
    g.add((municipality_uri, RDF.type, SCHEMA.City))
    g.add((province_uri, RDF.type, SCHEMA.AdministrativeArea))
    g.add((location_uri, RDF.type, SCHEMA.GeoCoordinates))

    # Add Data Properties
    g.add((beer_uri, SCHEMA.name, Literal(beer_name, datatype=XSD.string)))
    g.add((beer_uri, EX.alcoholPercentage, Literal(str(row["Alcohol Content"]), datatype=XSD.string)))
    g.add((beer_uri, EX.beerRating, Literal(row["Rating"], datatype=XSD.float)))
    g.add((beer_uri, EX.beerPopularity, Literal(row["Check-ins"], datatype=XSD.integer)))

    if pd.notna(row["Description"]):  # Avoid NaN issues
        g.add((beer_uri, EX.beerDescription, Literal(str(row["Description"]), datatype=XSD.string)))

    g.add((brewery_uri, SCHEMA.name, Literal(brewery_name, datatype=XSD.string)))
    g.add((municipality_uri, SCHEMA.name, Literal(municipality, datatype=XSD.string)))
    g.add((province_uri, SCHEMA.name, Literal(province, datatype=XSD.string)))

    # Add Relationships
    g.add((beer_uri, EX.brewedBy, brewery_uri))
    g.add((brewery_uri, EX.locatedInMunicipality, municipality_uri))
    g.add((municipality_uri, EX.isContainedInProvince, province_uri))
    g.add((beer_uri, EX.hasLocation, location_uri))

    if pd.notna(row["Latitude"]) and pd.notna(row["Longitude"]):
        g.add((location_uri, SCHEMA.latitude, Literal(row["Latitude"], datatype=XSD.float)))
        g.add((location_uri, SCHEMA.longitude, Literal(row["Longitude"], datatype=XSD.float)))

# Save as Turtle format
g.serialize("beers_graph.ttl", format="turtle")

print("RDF graph saved as beers_graph.ttl")


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/lpaggen/miniforge3/envs/scraping/lib/python3.13/site-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: could not convert string to float: 'Unknown'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#float, Converter=<class 'float'>
Traceback (most recent call last):
  File "/Users/lpaggen/miniforge3/envs/scraping/lib/python3.13/site-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
ValueError: could not convert string to float: 'Unknown'


RDF graph saved as beers_graph.ttl


/Users/lpaggen/miniforge3/envs/scraping/lib/python3.13/site-packages/rdflib/term.py:1620: UserWarning: Serializing weird numerical rdflib.term.Literal('Unknown', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float'))
  warnings.warn(f"Serializing weird numerical {self!r}")
